In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import acquire
import prepare

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
rawdf = acquire.get_zillow_data()

In [4]:
rawdf.head()

,longitude,latitude,bedrooms,bathrooms,square_feet,lot_size,has_pool,fips_code,age,assessed_value,tax_amount
0,"-118,217048","33,798657",4.00,2.00,"1,339.00","6,107.00",NaN,06037,40.00,"298,371.00","3,714.58"
1,"-118,531253","34,039467",2.00,2.00,"1,231.00","11,837.00",NaN,06037,67.00,"161,897.00","2,031.39"
2,"-117,613897","33,663005",5.00,3.00,"3,008.00","6,760.00",1.00,06059,22.00,"472,384.00","7,234.74"
3,"-118,393309","34,227422",2.00,1.00,996.00,"7,518.00",NaN,06037,77.00,"58,693.00",751.66
4,"-118,306261","33,745855",4.00,2.00,"1,329.00","7,005.00",NaN,06037,73.00,"62,194.00",886.96


In [5]:
df = prepare.prep_zillow(rawdf)

In [6]:
df.shape

(47123, 23)

In [7]:
df.head()

,longitude,latitude,bedrooms,bathrooms,square_feet,lot_size,has_pool,fips_code,age,assessed_value,...,bathrooms_size,county_Los Angeles,county_Orange,county_Ventura,bedrooms_size_small,bedrooms_size_medium,bedrooms_size_large,bathrooms_size_small,bathrooms_size_medium,bathrooms_size_large
0,"-118,217048","33,798657",4.00,2.00,"1,339.00","6,107.00",0.00,06037,40,"298,371.00",...,small,1,0,0,0,1,0,1,0,0
1,"-118,531253","34,039467",2.00,2.00,"1,231.00","11,837.00",0.00,06037,67,"161,897.00",...,small,1,0,0,1,0,0,1,0,0
2,"-117,613897","33,663005",5.00,3.00,"3,008.00","6,760.00",1.00,06059,22,"472,384.00",...,medium,0,1,0,0,0,1,0,1,0
3,"-118,393309","34,227422",2.00,1.00,996.00,"7,518.00",0.00,06037,77,"58,693.00",...,small,1,0,0,1,0,0,1,0,0
4,"-118,306261","33,745855",4.00,2.00,"1,329.00","7,005.00",0.00,06037,73,"62,194.00",...,small,1,0,0,0,1,0,1,0,0


## Simple Model
$$ y = mx + b $$

$$y = f(x)$$

In [8]:
scaled_df = df.copy()

In [9]:
scaler = MinMaxScaler()

In [42]:
scaler.fit(scaled_df[['square_feet', 'lot_size', 'age', 'bedrooms', 'bathrooms']])

MinMaxScaler()

In [43]:
scaled_df[['square_feet', 'lot_size', 'age', 'bedrooms', 'bathrooms']] = scaler.transform(scaled_df[['square_feet', 'lot_size', 'age', 'bedrooms', 'bathrooms']])

In [44]:
scaled_df.head()

,longitude,latitude,bedrooms,bathrooms,square_feet,lot_size,has_pool,fips_code,age,assessed_value,tax_amount,county,bedrooms_size,bathrooms_size,county_Los Angeles,county_Orange,county_Ventura,bedrooms_size_small,bedrooms_size_medium,bedrooms_size_large,bathrooms_size_small,bathrooms_size_medium,bathrooms_size_large
0,"-118,217048","33,798657",0.60,0.18,0.13,0.05,0.00,06037,0.28,"298,371.00","3,714.58",Los Angeles,medium,small,1,0,0,0,1,0,1,0,0
1,"-118,531253","34,039467",0.20,0.18,0.12,0.11,0.00,06037,0.48,"161,897.00","2,031.39",Los Angeles,small,small,1,0,0,1,0,0,1,0,0
2,"-117,613897","33,663005",0.80,0.36,0.40,0.06,1.00,06059,0.15,"472,384.00","7,234.74",Orange,large,medium,0,1,0,0,0,1,0,1,0
3,"-118,393309","34,227422",0.20,0.00,0.08,0.06,0.00,06037,0.55,"58,693.00",751.66,Los Angeles,small,small,1,0,0,1,0,0,1,0,0
4,"-118,306261","33,745855",0.60,0.18,0.13,0.06,0.00,06037,0.52,"62,194.00",886.96,Los Angeles,medium,small,1,0,0,0,1,0,1,0,0


In [45]:
s_train, s_validate, s_test = prepare.split(scaled_df)

In [46]:
pd.set_option('display.max_columns', None)

In [47]:
cols = ['square_feet', 'lot_size', 'has_pool', 'age', 'county_Los Angeles', 'county_Orange', 'bedrooms', 'bathrooms']

X_train = s_train[cols]
y_train = s_train.assessed_value

X_validate = s_validate[cols]
y_validate = s_validate.assessed_value

X_test = s_test[cols]
y_test = s_test.assessed_value

In [48]:
train_predictions = pd.DataFrame({
    'actual': s_train.assessed_value
}) 
validate_predictions = pd.DataFrame({
    'actual': s_validate.assessed_value
}) 

In [84]:
final_train_predictions = pd.DataFrame({
    'actual': s_train.assessed_value
}) 
final_validate_predictions = pd.DataFrame({
    'actual': s_validate.assessed_value
}) 
final_test_predictions = pd.DataFrame({
    'actual': s_test.assessed_value
})

## Multiple  Regression + RFE

In [49]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
k = 6

# 1. Transform our X
rfe = RFE(lm, n_features_to_select=5)
rfe.fit(X_train, y_train)
print('selected top 8 features:', X_train.columns[rfe.support_])
X_train_rfe = rfe.transform(X_train)

selected top 8 features: Index(['square_feet', 'lot_size', 'age', 'bedrooms', 'bathrooms'], dtype='object')


### Looping through k

In [50]:
for k in range(2,8):
    lm = LinearRegression()
    # 1. Transform our X
    rfe = RFE(lm, n_features_to_select=k)
    rfe.fit(X_train, y_train)
    # 2. Use the transformed x in our model
    X_train_rfe = rfe.transform(X_train)
    X_validate_rfe = rfe.transform(X_validate)
    lm.fit(X_train_rfe, y_train)
    
    # Make predictions
    train_predictions['multiple_rfe_k=', k] = lm.predict(X_train_rfe)
    validate_predictions['multiple_rfe_k=', k] = lm.predict(X_validate_rfe)
    k+=1

In [51]:
train_predictions.head()

,actual,"(multiple_rfe_k=, 2)","(multiple_rfe_k=, 3)","(multiple_rfe_k=, 4)","(multiple_rfe_k=, 5)","(multiple_rfe_k=, 6)","(multiple_rfe_k=, 7)"
14540,"471,024.00","284,478.18","296,936.86","294,389.84","294,708.55","341,281.65","336,950.48"
11215,"61,339.00","296,492.45","307,360.21","307,658.37","306,977.58","348,857.74","345,070.64"
27417,"75,483.00","358,554.10","373,228.26","372,143.25","371,398.82","415,978.05","412,265.89"
21049,"430,000.00","260,668.10","239,544.08","240,170.97","239,692.28","221,087.66","218,623.23"
25174,"619,000.00","300,861.27","274,414.91","278,273.31","281,313.47","272,432.94","268,301.14"


In [52]:
train_predictions['baseline'] = y_train.mean()

In [53]:
def calculate_rmse(y_predicted):
    return mean_squared_error(train_predictions.actual, y_predicted, squared = False)

train_predictions.apply(calculate_rmse).sort_values()

actual                       0.00
(multiple_rfe_k=, 7)   232,763.09
(multiple_rfe_k=, 6)   233,166.16
(multiple_rfe_k=, 5)   234,858.62
(multiple_rfe_k=, 4)   234,875.93
(multiple_rfe_k=, 3)   235,076.59
(multiple_rfe_k=, 2)   235,705.45
baseline               274,862.72
dtype: float64

In [54]:
validate_predictions['baseline'] = y_validate.mean()

In [55]:
validate_predictions.head()


,actual,"(multiple_rfe_k=, 2)","(multiple_rfe_k=, 3)","(multiple_rfe_k=, 4)","(multiple_rfe_k=, 5)","(multiple_rfe_k=, 6)","(multiple_rfe_k=, 7)",baseline
40104,"410,538.00","419,474.80","414,057.38","413,000.57","413,635.57","394,462.55","423,705.62","405,736.35"
18167,"339,849.00","376,005.02","376,343.82","378,991.27","380,363.94","362,945.37","393,570.22","405,736.35"
7121,"222,431.00","360,714.14","363,077.74","364,780.39","362,573.74","397,646.21","393,115.36","405,736.35"
30194,"857,504.00","387,825.21","361,887.72","364,578.94","365,770.98","353,101.33","349,126.13","405,736.35"
27143,"99,085.00","358,092.85","360,803.55","353,729.52","351,212.51","388,819.46","383,003.17","405,736.35"


In [56]:
def calculate_rmse(y_predicted):
    return mean_squared_error(validate_predictions.actual, y_predicted, squared = False)

validate_predictions.apply(calculate_rmse).sort_values()

actual                       0.00
(multiple_rfe_k=, 7)   230,221.87
(multiple_rfe_k=, 6)   230,727.85
(multiple_rfe_k=, 5)   232,270.96
(multiple_rfe_k=, 4)   232,304.06
(multiple_rfe_k=, 3)   232,661.45
(multiple_rfe_k=, 2)   233,345.88
baseline               272,468.83
dtype: float64

In [83]:
# k=7 is the best

230221-232763

-2542

**K=7 has the lowest rmse, highest performance**

In [86]:
# Adding k=7 to final table
lm = LinearRegression()
 # 1. Transform our X
    
rfe = RFE(lm, n_features_to_select=7)
rfe.fit(X_train, y_train)
# 2. Use the transformed x in our model
X_train_rfe = rfe.transform(X_train)
X_validate_rfe = rfe.transform(X_validate)
X_test_rfe = rfe.transform(X_test)
lm.fit(X_train_rfe, y_train)
    
# Make predictions
final_train_predictions['multiple_rfe_k=7'] = lm.predict(X_train_rfe)
final_validate_predictions['multiple_rfe_k=7'] = lm.predict(X_validate_rfe)
final_test_predictions['multiple_rfe_k=7']=lm.predict(X_test_rfe)

## Polynomial Features

In [57]:
train_pred = pd.DataFrame({
    'actual': s_train.assessed_value
}) 
validate_pred = pd.DataFrame({
    'actual': s_validate.assessed_value
}) 

In [58]:
from sklearn.preprocessing import PolynomialFeatures

# 1. Generate Polynomial Features, k=2
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)

In [59]:
lm = LinearRegression()
lm.fit(X_train_poly, y_train)
X_validate_poly = poly.transform(X_validate)
train_pred['baseline'] = y_train.mean()
validate_pred['baseline'] = y_validate.mean()
train_pred['polynomial degree 2'] = lm.predict(X_train_poly)
validate_pred['polynomial degree 2'] = lm.predict(X_validate_poly)
train_pred.head()

,actual,baseline,polynomial degree 2
14540,"471,024.00","409,241.54","285,097.77"
11215,"61,339.00","409,241.54","313,418.00"
27417,"75,483.00","409,241.54","333,493.35"
21049,"430,000.00","409,241.54","243,253.97"
25174,"619,000.00","409,241.54","337,785.11"


In [60]:
validate_pred.head()

,actual,baseline,polynomial degree 2
40104,"410,538.00","405,736.35","425,943.78"
18167,"339,849.00","405,736.35","401,633.18"
7121,"222,431.00","405,736.35","402,310.14"
30194,"857,504.00","405,736.35","363,816.89"
27143,"99,085.00","405,736.35","370,505.32"


In [61]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['polynomial degree 2'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['polynomial degree 2'], squared = False)

In [65]:
train_rmse_b = mean_squared_error(train_pred.actual,train_pred['baseline'], squared = False)
validate_rmse_b = mean_squared_error(validate_pred.actual,validate_pred['baseline'], squared = False)

In [62]:
train_rmse, validate_rmse

(226018.23613893724, 223545.9546342702)

In [63]:
validate_rmse-train_rmse

-2472.281504667044

In [66]:
train_rmse_b, validate_rmse_b

(274862.7167862455, 272468.8282290647)

In [67]:
# Interaction terms only

poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)
lm = LinearRegression()
lm.fit(X_train_poly, y_train)

X_validate_poly = poly.transform(X_validate)
train_pred['polynomial degree 2 only interaction'] = lm.predict(X_train_poly)
validate_pred['polynomial degree 2 only interaction'] = lm.predict(X_validate_poly)
train_pred.head()

,actual,baseline,polynomial degree 2,polynomial degree 2 only interaction
14540,"471,024.00","409,241.54","285,097.77","293,158.68"
11215,"61,339.00","409,241.54","313,418.00","319,185.77"
27417,"75,483.00","409,241.54","333,493.35","331,075.64"
21049,"430,000.00","409,241.54","243,253.97","233,450.18"
25174,"619,000.00","409,241.54","337,785.11","328,970.75"


In [68]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['polynomial degree 2 only interaction'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['polynomial degree 2 only interaction'], squared = False)
train_rmse, validate_rmse

(226317.07147912346, 223939.20747348687)

In [69]:
validate_rmse-train_rmse

-2377.8640056365984

In [70]:
# k=3
poly = PolynomialFeatures(degree=3, include_bias=False, interaction_only=False)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)

In [71]:
lm = LinearRegression()
lm.fit(X_train_poly, y_train)
X_validate_poly = poly.transform(X_validate)
train_pred['baseline'] = y_train.mean()
validate_pred['baseline'] = y_validate.mean()
train_pred['polynomial degree 3'] = lm.predict(X_train_poly)
validate_pred['polynomial degree 3'] = lm.predict(X_validate_poly)
train_pred.head()

,actual,baseline,polynomial degree 2,polynomial degree 2 only interaction,polynomial degree 3
14540,"471,024.00","409,241.54","285,097.77","293,158.68","299,474.65"
11215,"61,339.00","409,241.54","313,418.00","319,185.77","308,079.89"
27417,"75,483.00","409,241.54","333,493.35","331,075.64","335,684.56"
21049,"430,000.00","409,241.54","243,253.97","233,450.18","263,100.74"
25174,"619,000.00","409,241.54","337,785.11","328,970.75","337,630.30"


In [72]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['polynomial degree 3'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['polynomial degree 3'], squared = False)
train_rmse, validate_rmse

(223633.46235207212, 222070.91872805462)

In [73]:
# k=4
poly = PolynomialFeatures(degree=4, include_bias=False, interaction_only=False)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)

In [74]:
lm = LinearRegression()
lm.fit(X_train_poly, y_train)
X_validate_poly = poly.transform(X_validate)
train_pred['baseline'] = y_train.mean()
validate_pred['baseline'] = y_validate.mean()
train_pred['polynomial degree 4'] = lm.predict(X_train_poly)
validate_pred['polynomial degree 4'] = lm.predict(X_validate_poly)
train_pred.head()

,actual,baseline,polynomial degree 2,polynomial degree 2 only interaction,polynomial degree 3,polynomial degree 4
14540,"471,024.00","409,241.54","285,097.77","293,158.68","299,474.65","309,613.66"
11215,"61,339.00","409,241.54","313,418.00","319,185.77","308,079.89","318,497.96"
27417,"75,483.00","409,241.54","333,493.35","331,075.64","335,684.56","302,993.86"
21049,"430,000.00","409,241.54","243,253.97","233,450.18","263,100.74","258,815.31"
25174,"619,000.00","409,241.54","337,785.11","328,970.75","337,630.30","340,044.95"


In [75]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['polynomial degree 4'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['polynomial degree 4'], squared = False)
train_rmse, validate_rmse

(221616.71599597967, 225548.63048489933)

**K=3 has the lowest rmse**

In [88]:
# Adding k=3 to final table
poly = PolynomialFeatures(degree=3, include_bias=False, interaction_only=False)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)
lm = LinearRegression()
lm.fit(X_train_poly, y_train)
X_validate_poly = poly.transform(X_validate)
X_test_poly = poly.transform(X_test)
final_train_predictions['baseline'] = y_train.mean()
final_validate_predictions['baseline'] = y_validate.mean()
final_test_predictions['baseline'] = y_test.mean()
final_train_predictions['polynomial degree 3'] = lm.predict(X_train_poly)
final_validate_predictions['polynomial degree 3'] = lm.predict(X_validate_poly)
final_test_predictions['polynomial degree 3'] = lm.predict(X_test_poly)

## Lasso-Lars

In [76]:
from sklearn.linear_model import LassoLars
# create the model object
lars = LassoLars(alpha=0)

# fit the model to our training data
lars.fit(X_train, y_train)

# predict validate
X_train_pred_lars = lars.predict(X_train)

pd.Series(lars.coef_, index=X_train.columns).sort_values()

bedrooms              -269,204.77
lot_size              -117,309.33
county_Los Angeles     -54,913.82
county_Orange           11,562.02
has_pool                36,232.93
age                     83,563.09
bathrooms              219,997.52
square_feet          1,170,303.06
dtype: float64

In [77]:
# create the model object
lars = LassoLars(alpha=1)

# fit the model to our training data
lars.fit(X_train, y_train)

# predict validate
X_train_pred_lars = lars.predict(X_train)
X_validate_pred_lars = lars.predict(X_validate)
# Add lassolars predictions to our predictions DataFrame
train_pred['lasso_lars'] = X_train_pred_lars
validate_pred['lasso_lars'] = X_validate_pred_lars

In [78]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['lasso_lars'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['lasso_lars'], squared = False)
train_rmse, validate_rmse

(232745.2303032925, 230203.61065288217)

In [79]:
validate_rmse-train_rmse

-2541.6196504103427

In [90]:
# Adding to final table
# create the model object
lars = LassoLars(alpha=1)

# fit the model to our training data
lars.fit(X_train, y_train)

# predict validate
X_train_pred_lars = lars.predict(X_train)
X_validate_pred_lars = lars.predict(X_validate)
X_test_pred_lars = lars.predict(X_test)
# Add lassolars predictions to our predictions DataFrame
final_train_predictions['lasso_lars'] = X_train_pred_lars
final_validate_predictions['lasso_lars'] = X_validate_pred_lars
final_test_predictions['lasso_lars'] = X_test_pred_lars

## Generalized Linear Model

In [80]:
from sklearn.linear_model import TweedieRegressor

# create the model object
glm = TweedieRegressor(power=1, alpha=0)

# fit the model to our training data
glm.fit(X_train, y_train)

# predict train
X_train_predict_glm = glm.predict(X_train)
X_validate_predict_glm = glm.predict(X_validate)
# Add lassolars predictions to our predictions DataFrame
train_pred['glm'] = X_train_predict_glm
validate_pred['glm'] = X_validate_predict_glm

In [81]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['glm'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['glm'], squared = False)
train_rmse, validate_rmse

(235321.89586322848, 233617.59325644202)

In [82]:
validate_rmse-train_rmse

-1704.3026067864557

In [91]:
# Adding to final table

glm = TweedieRegressor(power=1, alpha=0)

# fit the model to our training data
glm.fit(X_train, y_train)

# predict train
X_train_predict_glm = glm.predict(X_train)
X_validate_predict_glm = glm.predict(X_validate)
X_test_predict_glm = glm.predict(X_test)
# Add lassolars predictions to our predictions DataFrame
final_train_predictions['glm'] = X_train_predict_glm
final_validate_predictions['glm'] = X_validate_predict_glm
final_test_predictions['glm'] = X_test_predict_glm

## Evaluation

In [94]:
def calculate_rmse(y_predicted):
    return mean_squared_error(final_train_predictions.actual, y_predicted, squared = False)

final_train_predictions.apply(calculate_rmse).sort_values()

actual                      0.00
polynomial degree 3   223,633.46
lasso_lars            232,745.23
multiple_rfe_k=7      232,763.09
glm                   235,321.90
baseline              274,862.72
dtype: float64

In [95]:
def calculate_rmse(y_predicted):
    return mean_squared_error(final_validate_predictions.actual, y_predicted, squared = False)

final_validate_predictions.apply(calculate_rmse).sort_values()

actual                      0.00
polynomial degree 3   222,070.92
lasso_lars            230,203.61
multiple_rfe_k=7      230,221.87
glm                   233,617.59
baseline              272,468.83
dtype: float64

In [96]:
def calculate_rmse(y_predicted):
    return mean_squared_error(final_test_predictions.actual, y_predicted, squared = False)

final_test_predictions.apply(calculate_rmse).sort_values()

actual                      0.00
polynomial degree 3   225,323.49
lasso_lars            233,614.86
multiple_rfe_k=7      233,644.57
glm                   237,682.56
baseline              277,387.85
dtype: float64

In [97]:
(277387.85-225323.49)/277387.85

0.18769517121964782